In [1]:

# Install necessary libraries
!pip install numpy pandas scikit-learn nltk

In [2]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import string

In [4]:

# Load the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
df = pd.read_csv(url, sep='\t', names=['label', 'message'])

# Display the first few rows
df.head()

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['SMSSpamCollection', 'readme']

In [5]:

import zipfile
import urllib.request

# Download the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/sm

SyntaxError: unterminated string literal (detected at line 5) (<ipython-input-5-389b3d9fc8a1>, line 5)

In [6]:

import zipfile
import urllib.request

# Download the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
filename = 'smsspamcollection.zip'
urllib.request.urlretrieve(url, filename)

# Unzip the file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

# Load the specific file
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'message'])

# Display the first few rows
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:

# Download stopwords from nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Text preprocessing function
def preprocess_text(text):
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenize and remove stopwords
    words = [word for word in text.split() if word.lower() not in stop_words]
    return ' '.join(words)

# Apply the preprocessing function to the dataset
df['cleaned_message'] = df['message'].apply(preprocess_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,label,message,cleaned_message
0,ham,"Go until jurong point, crazy.. Available only ...",Go jurong point crazy Available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,U dun say early hor U c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah dont think goes usf lives around though


In [8]:

# Convert text into numerical features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_message'])

# Encode labels (spam = 1, ham = 0)
df['label_encoded'] = df['label'].map({'ham': 0, 'spam': 1})

# Define target variable
y = df['label_encoded']

In [9]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:

# Train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [11]:

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9748878923766816

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       966
           1       0.88      0.95      0.91       149

    accuracy                           0.97      1115
   macro avg       0.93      0.96      0.95      1115
weighted avg       0.98      0.97      0.98      1115



In [12]:

# Function to predict new email
def predict_spam(email):
    email_processed = preprocess_text(email)
    email_vectorized = vectorizer.transform([email_processed])
    prediction = model.predict(email_vectorized)
    return 'Spam' if prediction == 1 else 'Ham'

# Test with a new email
new_email = "Congratulations! You've won a $1000 gift card. Click here to claim it."
print(f"Prediction: {predict_spam(new_email)}")

Prediction: Spam
